In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## [DMatch类](https://www.bbsmax.com/A/obzbr0x0zE/)

1、2、3不用说，是三个构造函数。接着, 

**int queryIdx** –>是**测试图像**(图像2)的特征点描述符（descriptor）的下标，同时也是描述符对应特征点（keypoint)的下标。

**int trainIdx** –> 是**样本图像**(图像1)的特征点描述符的下标，同样也是相应的特征点的下标。

**int imgIdx** –>当样本是多张图像的话有用。

**float distance** –>代表这一对匹配的特征点描述符（本质是向量）的**欧氏距离**，数值越小也就说明两个特征点越相像。

最后， 也就是一个小于操作符的重载，用于比较和排序。 比较的是上述的distance，当然是越小越好。

## [KeyPoints类](https://www.bbsmax.com/A/obzbr0x0zE)


In [2]:
# AKAZE特征点检测和匹配
def GetImageMatches(img1,img2):
    #检测特征点并返回特征点keypoints 和特征点描述符descriptors
    akaze = cv2.AKAZE_create()
    kp1, desc1 = akaze.detectAndCompute(img1,None)
    kp2, desc2 = akaze.detectAndCompute(img2,None)
    
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck = True)#crossCheck=True: 两个特征点之间距离须互为最近
    matches = matcher.match(desc1, desc2)#返回最佳匹配特征点对matches(DMatch类)
    
    matches = sorted(matches, key = lambda x:x.distance)#按照距离将特征点对升序排列
    #Dmatch.(float) distance 代表这一对匹配的特征点描述符（本质为向量）的欧式距离，数值越小说明两个特征点越像
    
    # 取前80%匹配对
    good = []
    #print(matches[int(len(matches)*0.8)])
    for n in range (int(len(matches)*0.9)):
        good.append(matches[n])
    return kp1,desc1,kp2,desc2,good

if __name__ == '__main__':
    img1 = cv2.imread("hat_L.jpg")#检测到209个特征点，注意图片后缀
    img2 = cv2.imread("hat_R.jpg")#检测到532个特征点
    kp1,desc1,kp2,desc2,matches = GetImageMatches(img1,img2)
    
    #显示keypoints
    #必须要先初始化img2
    img_ = img1.copy()
    img_ = cv2.drawKeypoints(img1, kp1, img_, color=(0,255,0))
    while cv2.waitKey(100) != 27:
        cv2.imshow('Detected AKAZE keypoints', img_)
    
    #最后是连线可视化
    res = cv2.drawMatches(img1, kp1, img2, kp2, matches[:], None, flags=2)
    cv2.imshow('res',res)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
# 得到2个图像中匹配点的图像坐标
def GetAlignedMatches(kp1,desc1,kp2,desc2,matches):
    # 防止万一matches 列表还没排序：
    matches = sorted(matches, key = lambda x:x.distance) #内建函数sorted按照元素的distance排序，返回一个新list
    
    # 从matches中取回两个图像的keypoints的对应的序号
    img1idx = np.array([m.queryIdx for m in matches]) #图像2的特征点下标，将列表转换成ndarry（方便运算）
    img2idx = np.array([m.trainIdx for m in matches]) #图像1的特征点下标
    
    # 取出匹配的keypoints（matches中元素数量 < 各自点集元素数量）
    kp1_ = (np.array(kp1))[img1idx]
    kp2_ = (np.array(kp2))[img2idx]
    
    # 取回匹配的keypoints的图像坐标
    img1pts = np.array([kp.pt for kp in kp1_])
    img2pts = np.array([kp.pt for kp in kp2_])
    
    return img1pts,img2pts,img1idx,img2idx

## numpy.repeat 和 numpy.tile
repeat 和 rile是属于ndarray对象的方法。

**ndarray**
1. ndarray.ndim：数组的维度，即数组轴（axes）的个数，其数量等于秩（rank）。
2. ndarray.shape：各维度的大小

**repeat**可以通过2个管道使用：(1)numpy.repeat(a,repeats,axis=None); (2)object(ndarray).repeat(repeats,axis=None)
* axis = None，先行后列，逐个元素复制repeats次，形成一个行向量
* axis=0,沿着y轴复制，实际上增加行数，列数不变，各行复制repeats次
* axis=1,沿着x轴复制，实际上增加列数，行数不变，各列复制repeats次
* repeats可以为一个数，也可以为一个矩阵，各元素代表每行/列复制几次（[2,3]意味着第一行/列重复2次，第二行/列重复3次，行/列由axis决定）

**tile** 使用：numpy.tile(A,reps)重复数组A来构建新数组

*A是待输入数组，reps 决定A沿着各个维度重复的次数。

1. reps.ndim > A.ndim ：向A中添加新轴扩充A的维度。一般通过向shape对应的元组中添加1完成对A维度的扩充。扩充完成后,根据reps值对A中相应维度的值进行重复
2. reps.ndim < A.ndim ：将reps扩充至与A相同的维度。也是向shape对应的元组的左侧添1：(3,),(1,3),(1,1,3)，然后再进行重复，重复整个矩阵。
3. reps.ndim = A.ndim ：不需要扩充，直接按reps的值对相应维度的值进行重复

## numpy.linalg.svd 函数
函数：np.linalg.svd(a,full_matrices=1,compute_uv=1)。
* a 是一个形如（M,N）的矩阵
* full_matrices 的取值是为0 或者 1，默认为1，这时U 的大小为（M，M），V的大小为（N，N）。如果full_matrices不是1，则u的大小为(M,K)，v的大小为(K,N) ，K=min(M,N)。

返回值：u,s,v
* u大小为(M,M)，s大小为(M,N)，v大小为(N,N)。
* A = u*s*v
* s是对矩阵a的**奇异值分解**。s除了对角元素不为0，其他元素都为0，并且对角元素从大到小排列。s中有n个奇异值，一般排在后面的比较接近0，所以仅保留比较大的r个奇异值。 

In [4]:
# 由8点估算基本矩阵F
def EstimateFundamentalMatrix(img1_8pts,img2_8pts):#输入两幅图像中的8个点
    #如果不是就变成齐次坐标
    if img1_8pts.shape[1] == 2: #如果图像每个特征点只有2个坐标
        img1_8pts = cv2.convertPointsToHomogeneous(img1_8pts)[:,0,:]#变成齐次坐标：添1，在第二维增加一维，成为3维坐标，.shape = (141,1,3)
        img2_8pts = cv2.convertPointsToHomogeneous(img2_8pts)[:,0,:]
        #加[:,0,:]的作用是在第二维上减一维，变成：shape=(141,3)：加一个0减一维；如果0换成None，就是在该维度上加一维
        
    A = np.zeros((img1_8pts.shape[0],9)) #定义一个：图像1的第1维数(8) x 9 的全0矩阵A
    
    img1_8pts_ = img1_8pts.repeat(3,axis=1) #axis=1,各列复制3次（[111222333]），得到 img1pts_.shape=(8,9）
    img2_8pts_ = np.tile(img2_8pts,(1,3)) #行方向不重复，列方向上矩阵重复3次[123 123 123]，得到 img2pts_.shape=(8,9)
    
    A = img1_8pts_ * img2_8pts_ #各元素对位相乘,A.shape=(8,9)
    
    u,s,v = np.linalg.svd(A) #线性代数.奇异值分解
    F = v[-1,:].reshape((3,3),order='F') #取v的最后一行reshape，按优先列读写
    
    u,s,v = np.linalg.svd(F)
    F = u.dot(np.diag(s).dot(v)) #矩阵内积
    
    F = F / F[-1,-1]
    
    return F

In [5]:
# test cell
# data = mat([[1,2,3],[4,5,6]])
img1_8pts= np.array([[397.51690674, 207.14871216, 1.], [345.48895264, 198.8712616 ,  1.        ],  [228.76841736, 244.31945801,  1.  ],  [346.37963867, 141.98188782,  1.        ], [359.28860474, 242.01344299,  1.        ],  [336.2354126 , 190.75994873,  1.        ], [251.89804077, 148.5166626 ,  1.        ],  [331.28744507, 239.37399292,  1.        ]])
img2_8pts= np.array([[320.29403687, 164.46336365,  1. ],  [264.3164978 , 156.3860321, 1. ], [130.02644348, 137.49526978,   1.        ],  [268.4407959 ,  98.52967834,  1.        ],  [275.8727417 , 199.86174011,  1.        ],  [254.89256287, 148.70428467,  1.        ],  [169.9493866 , 105.59056854,  1. ],  [246.43341064 , 197.26600647,   1.        ]])

A = np.zeros((img1_8pts.shape[0],9))

img1_8pts_ = img1_8pts.repeat(3,axis=1)
img2_8pts_ = np.tile(img2_8pts,(1,3))
# print(img1_8pts_)
# print(img2_8pts_)
product = np.multiply(img1_8pts_, img2_8pts_)
# a=array(([1,2,3],[4,5,6]))
# b = array(([7,8,9],[10,11,12]))
# c = a*b
# print(a)
# print(b)
# print(c)
# U,sigma,VT = np.linalg.svd(data)
# print(U)
# print(sigma)
# print (VT)

In [6]:
# 计算抽样误差
def SampsonError(F,x1,x2):
    num = np.sum(x1.dot(F) * x2, axis=-1) # 点集x1内积上F再与点集x2对位相乘，
    
    F_src = np.dot(F, x1.T)
    Ft_dst = np.dot(F.T, x2.T)
    
    dst_F_src = np.sum(x2 * F_src.T, axis=1)
    
    return np.abs(dst_F_src)/np.sqrt(F_src[0]**2 + F_src[1]**2 + Ft_dst**2)

In [7]:
# RANSAC估计基本矩阵
def EstimateFundamentalMatrixRANSAC(img1pts, img2pts, outlierThres, prob=None, iters=None):
    if img1pts.shape[1]==2: #如果是2列，说明还不是齐次坐标，需要添加1
        img1pts = cv2.convertPointsToHomogeneous(img1pts)[:,0,:]
        img2pts = cv2.convertPointsToHomogeneous(img2pts)[:,0,:]
    
    bestInliers, bestF, bestmask = 0, None, None #定义最好的内点个数，最好的F，最好的组合
    
    for i in range(iters):
        # 随机选8个点
        mask = np.random.randint(low=0,high=img1pts.shape[0],size=(8,))
        img1ptsiter = img1pts[mask]
        img2ptsiter = img2pts[mask]
#         print("img1ptsiter=",img1ptsiter)
#         print("img2ptsiter=",img2ptsiter)
        
        # 估计基本矩阵，并评估误差
        Fiter = EstimateFundamentalMatrix(img1ptsiter,img2ptsiter)
        error = SampsonError(Fiter,img1pts,img2pts) 
        qualified = error < outlierThres #error小于outlierThres 的点为True
        numInliers = np.sum(qualified) #统计True的个数
        
        # 更新最佳测量结果
        if bestInliers < numInliers:
            bestInliers = numInliers
            bestF = Fiter
            bestmask = mask #？？？？？？？？？？？
            
    # 拟合所有找到的内点上的最小二乘法
    F = EstimateFundamentalMatrix(img1pts[bestmask], img2pts[bestmask])#???
    
    return F,bestmask

In [8]:
# 用SVD方法将本质矩阵E分解为R 和 t
def ExtractCameraPoses(E):
    u,d,v = np.linalg.svd(E)
    W = np.array(([0,-1,0], [1,0,0], [0,0,1]))
    
    Rs, Cs = np.zeros((4,3,3)), np.zeros((4,3))
    
    t = u[:,-1]
    R1 = u.dot(W.dot(v))
    R2 = u.dot(W.T.dot(v))
    
    if np.linalg.det(R1) < 0:
        R1 = R1 * -1
    
    if np.linalg.det(R2) < 0:
        R2 = R2 * -1
        
    return R1,R2,t

In [9]:
# 三角测量：直接线性方法（投影方程）
def GetTriangulatedPts(img1pts,img2pts,K,R,t,triangulateFunc): 
    img1ptsHom = cv2.convertPointsToHomogeneous(img1pts)[:,0,:]
    img2ptsHom = cv2.convertPointsToHomogeneous(img2pts)[:,0,:]

    img1ptsNorm = (np.linalg.inv(K).dot(img1ptsHom.T)).T
    img2ptsNorm = (np.linalg.inv(K).dot(img2ptsHom.T)).T

    img1ptsNorm = cv2.convertPointsFromHomogeneous(img1ptsNorm)[:,0,:]
    img2ptsNorm = cv2.convertPointsFromHomogeneous(img2ptsNorm)[:,0,:]
    
    pts4d = triangulateFunc(np.eye(3,4),np.hstack((R,t)),img1ptsNorm.T,img2ptsNorm.T)
    pts3d = cv2.convertPointsFromHomogeneous(pts4d.T)[:,0,:]
    
    return pts3d

In [10]:
# 把3D点坐标转换成meshlab适合的坐标
def pts2ply(pts,filename = 'out.ply'):
    f = open(filename,'w')
    f.write('ply\n')
    f.write('format ascii 1.0\n')
    f.write('element vertex {}\n'.format(pts.shape[0]))
    
    f.write('property float x\n')
    f.write('property float y\n')
    f.write('property float z\n')
    
    f.write('property uchar red\n')
    f.write('property uchar green\n')
    f.write('property uchar blue\n')
    
    f.write('end_header\n')
    
    for pt in pts: 
        f.write('{} {} {} 255 255 255\n'.format(pt[0],pt[1],pt[2]))
    f.close()

# 主函数：

In [28]:
# 主函数
import numpy as np
import cv2

#Reading two images for reference
img1 = cv2.imread("hat_L.jpg")#检测到258个特征点
img2 = cv2.imread("hat_R.jpg")#检测到284个特征点

#Converting from BGR to RGB format
img1 = img1[:,:,::-1]
img2 = img2[:,:,::-1]

# AKAZE特征点检测和匹配
kp1,desc1,kp2,desc2,matches = GetImageMatches(img1,img2)
# 对齐两个keypoints向量
img1pts,img2pts,img1idx,img2idx = GetAlignedMatches(kp1,desc1,kp2,desc2,matches)

# 计算基本矩阵——RANSAC
F, mask = EstimateFundamentalMatrixRANSAC(img1pts,img2pts,0.1,iters=20000)

In [29]:
# 由F求E
K = np.array(([678.48, 0, 302.81], [0, 678.50, 225.80], [0, 0, 1]))
#K = np.array(([681.12, 0, 303.78], [0, 680.58, 223.50], [0, 0, 1]))
E = K.T.dot(F.dot(K))
print(E)

# 用SVD方法将本质矩阵E分解为R 和 t
R1,R2,t = ExtractCameraPoses(E)
t = t[:,np.newaxis]

# 三角测量：直接线性方法（投影方程）得到特征点的三维坐标
pts3d = GetTriangulatedPts(img1pts,img2pts,K,R2,t,cv2.triangulatePoints)

# 把3D点坐标转换成meshlab适合的坐标
pts2ply(pts3d,'castle_2view.ply')

[[ -0.09010957  -6.33839553  -0.50945662]
 [  6.21103452   0.48415712  15.39172887]
 [  0.94337908 -16.24269927   0.89371002]]


In [16]:
# 得到P
# 合并R，t
outArgu1 = np.column_stack((R1,t))
outArgu2 = np.column_stack((R2,t))

P1 = np.dot(K,outArgu1)
P2 = np.dot(K,outArgu2)

print(P1)
print(P2)

[[ 6.85494340e+02 -2.70689699e+02  9.40935909e+01  7.20292706e+02]
 [-1.34061255e+02 -6.67792488e+02 -2.17781228e+02 -7.82865846e+01]
 [ 4.95990147e-01 -1.59328162e-01 -8.53585561e-01  2.74682805e-01]]
[[ 6.86097978e+02  4.15408334e+01  2.82086743e+02  7.20292706e+02]
 [-1.50132490e+01  6.92698032e+02  1.76895247e+02 -7.82865846e+01]
 [ 2.70650235e-02  7.19425032e-02  9.97041504e-01  2.74682805e-01]]


有opencv库内置函数？？算坐标

help(cv2.)

后面有2条路：
1. Bundle Adjustment
2. 多视图